In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt

In [ ]:
import gurobipy as gp
from gurobipy import Env

env = Env(empty=True)
# License credentials loaded from environment variables or local file
# env.setParam('WLSACCESSID', '...')
env.start()


In [ ]:
def generate_data():
    # 1. Load Data
    N = 20
    df_full = pd.read_csv("uscities.csv")
    df_nodes = df_full.sort_values(by='population', ascending=False).head(N).copy()
    df_nodes = df_nodes[['city', 'lat', 'lng', 'population']].rename(
        columns={'city': 'City', 'lat': 'Lat', 'lng': 'Lon', 'population': 'Population'}
    )
    
    nodes = df_nodes['City'].tolist()
    
    # 2. Initial Demand Calculation
    demand = {city: -pop / 1000 for city, pop in zip(nodes, df_nodes['Population'])}
    
    # 3. Add Supply (Factories)
    total_demand = abs(sum(demand.values()))
    if 'New York' in demand: demand['New York'] += total_demand * 0.47
    if 'Los Angeles' in demand: demand['Los Angeles'] += total_demand * 0.26
    if 'Chicago' in demand: demand['Chicago'] += total_demand * 0.27
    
    # 4. CRITICAL FIX: Round first, THEN balance
    # First, round everyone to 2 decimals
    demand = {k: round(v, 2) for k, v in demand.items()}
    
    # Calculate the tiny error introduced by rounding
    current_sum = sum(demand.values())
    
    # Force the correction city to absorb the error so sum is EXACTLY 0.00000
    correction_city = 'New York' if 'New York' in demand else nodes[0]
    demand[correction_city] -= current_sum
    
    # Final check (Optional debug print)
    print(f"Data Generated. Total Demand Sum: {sum(demand.values())} (Should be 0.0)")

    modes_info = {
        'Van':   {'cost': 2.50, 'co2': 0.24, 'cap': 300,    'fixed': 200},
        'Truck': {'cost': 2.2, 'co2': 0.40, 'cap': 8000,   'fixed': 2500}, # Boosted to 5k
        'Train': {'cost': 1.8, 'co2': 0.05, 'cap': 100000,  'fixed': 25000} # Boosted to 50k
    }
    
    return df_nodes, demand, modes_info

In [ ]:
# Helper function to calculate distance
def get_dist(lat1, lon1, lat2, lon2):
    # Haversine formula
    r = 3956 # Miles
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return 2 * r * asin(sqrt(a))


In [ ]:
# =============================================================================
# SECTION 2: METHOD 1 - BRANCH AND CUT (GUROBI) 
# =============================================================================

def solve_logistics_network(df_nodes, demand, modes_info, env, E_max=10000000, clean_quota=0.3):
    
    # 1. Prepare Data Structures
    nodes = df_nodes['City'].tolist()
    modes = list(modes_info.keys())
    clean_modes = ['Van', 'Train'] 
    
    # We will build 'arcs' dynamically now based on the distance constraint
    valid_arcs_modes = [] 
    
    # Calculate Distances & Link Parameters
    dist = {}
    costs = {}
    emissions = {}
    fixed_costs = {}
    capacities = {}
    
    for i in nodes:
        for j in nodes:
            if i == j: continue
            
            # Get coords
            r1 = df_nodes[df_nodes['City'] == i].iloc[0]
            r2 = df_nodes[df_nodes['City'] == j].iloc[0]
            d_ij = get_dist(r1['Lat'], r1['Lon'], r2['Lat'], r2['Lon'])
            dist[(i,j)] = d_ij
            
            for mode in modes:
                # --- NEW CONSTRAINT: REALISM FILTER ---
                # Constraint: Trains cannot be used for short distances (< 500 miles). 
                # This forces the model to use Vans or Trucks for local deliveries (e.g., NY -> Brooklyn).
                if mode == 'Train' and d_ij < 500:
                    continue 
                
                # If valid, add to list of active arcs for this mode
                valid_arcs_modes.append((i, j, mode))
                
                # Calculate Parameters
                costs[(i,j,mode)] = modes_info[mode]['cost'] * d_ij
                emissions[(i,j,mode)] = modes_info[mode]['co2'] * d_ij
                fixed_costs[(i,j,mode)] = modes_info[mode]['fixed']
                capacities[(i,j,mode)] = modes_info[mode]['cap']

    # 2. Initialize Model (Passing the Env for the license)
    model = gp.Model("GreenLogistics_Method1", env=env)
    
    # 3. Decision Variables
    # We only create variables for "valid" (i,j,mode) tuples
    x = model.addVars(valid_arcs_modes, vtype=GRB.CONTINUOUS, name="flow")
    y = model.addVars(valid_arcs_modes, vtype=GRB.BINARY, name="open")
    
    # 4. Objective Function
    model.setObjective(
        gp.quicksum(fixed_costs[i,j,mode] * y[i,j,mode] + costs[i,j,mode] * x[i,j,mode] 
                    for i,j,mode in valid_arcs_modes),
        GRB.MINIMIZE
    )
    
    # 5. Constraints
    
    # (1) Flow Balance
    # Note: We must be careful to only sum over existing variables
    for i in nodes:
        flow_out = gp.quicksum(x[i,j,mode] for j in nodes for mode in modes if (i,j,mode) in valid_arcs_modes)
        flow_in  = gp.quicksum(x[j,i,mode] for j in nodes for mode in modes if (j,i,mode) in valid_arcs_modes)
        
        model.addConstr(flow_out - flow_in == demand[i], name=f"Balance_{i}")
        
    # (2) Capacity & Linking (Big-M)
    for i,j,mode in valid_arcs_modes:
        model.addConstr(x[i,j,mode] <= capacities[i,j,mode] * y[i,j,mode], 
                        name=f"Cap_{i}_{j}_{mode}")
            
    # (3) Global Emissions Cap
    model.addConstr(
        gp.quicksum(emissions[i,j,mode] * x[i,j,mode] for i,j,mode in valid_arcs_modes) <= E_max,
        name="Global_Emissions"
    )
    
    # (4) Clean Transport Quota
    total_flow = gp.quicksum(x[i,j,mode] for i,j,mode in valid_arcs_modes)
    clean_flow = gp.quicksum(x[i,j,mode] for i,j,mode in valid_arcs_modes if mode in clean_modes)
    
    model.addConstr(clean_flow >= clean_quota * total_flow, name="Clean_Quota")
    
    # 6. Optimize
    model.optimize()
    
    return model, x, y, dist

In [ ]:
def compute_baseline_stats(df_nodes, demand, modes_info, env):
    """
    1) Runs an unconstrained baseline (no binding emissions cap, no clean quota)
    2) Returns baseline emissions E0 and baseline clean share alpha0.
    """
    print("--- Computing Unconstrained Baseline ---")
    
    # 1. Solve baseline with loose constraints
    # E_max = 1e20 (Infinity), Clean Quota = 0.0
    model0, x0, y0, dist0 = solve_logistics_network(
        df_nodes, demand, modes_info, env,
        E_max=1e20,     
        clean_quota=0.0 
    )
    
    # 2. Calculate E0 (Total Emissions) and Alpha0 (Clean Share) from results
    E0 = 0
    total_flow0 = 0
    clean_flow0 = 0
    
    # Iterate directly over the solution variables
    for (i, j, m), var in x0.items():
        val = var.X
        if val > 0.001:
            d = dist0[(i,j)]
            E0 += val * d * modes_info[m]['co2']
            total_flow0 += val
            if m in ['Van', 'Train']:
                clean_flow0 += val
                
    alpha0 = clean_flow0 / total_flow0 if total_flow0 > 0 else 0.0
    
    return E0, alpha0, model0

In [ ]:
# =============================================================================
# SECTION 3: RUN THREE POLICY SCENARIOS 
# =============================================================================

if __name__ == "__main__":
    # 1. Generate data
    df_nodes, demand, modes_info = generate_data()

    # 2. Baseline stats
    E0, alpha0, baseline_model = compute_baseline_stats(df_nodes, demand, modes_info, env)
    
    print(f"\nBaseline Results:")
    print(f"  Emissions (E0): {E0:,.0f} kg")
    print(f"  Clean Share (a0): {alpha0:.1%}")
    print(f"  Min Cost:       ${baseline_model.objVal:,.2f}")
    
    # 3. Define scenarios
    # Note: We cap the 'clean_quota' at 0.90 to avoid infeasibility if alpha0 is high
    scenarios = {
        "1_Mild": {
            "E_max":       1.0 * E0,                  
            "clean_quota": min(max(alpha0, 0.30), 0.95) # Cap at 95%
        },
        "2_Target": {
            "E_max":       0.90 * E0, # Changed to 10% reduction (easier)                
            "clean_quota": 0.50                      
        },
        "3_Aggressive": {
            "E_max":       0.80 * E0, # Changed to 20% reduction (easier)                
            "clean_quota": 0.70                      
        }
    }
    
    results = {}
    
    # 4. Run each scenario
    for name, params in scenarios.items():
        print(f"\n" + "="*60)
        print(f"SCENARIO: {name}")
        print(f"Constraint: E_max <= {params['E_max']:,.0f} | Quota >= {params['clean_quota']:.0%}")
        print("-" * 60)
        
        model_s, x_s, y_s, dist_s = solve_logistics_network(
            df_nodes, demand, modes_info, env,
            E_max=params["E_max"],
            clean_quota=params["clean_quota"]
        )
        
        if model_s.status == GRB.OPTIMAL:
            # Recompute actuals
            E_s = 0
            total_flow_s = 0
            clean_flow_s = 0
            
            # --- PRINT ACTIVE ROUTES (Just like Method 1) ---
            print(f"{'Origin':<15} {'Dest':<15} {'Mode':<8} {'Flow (Tons)':<12} {'Dist':<8}")
            print("-" * 65)

            for (i, j, m), var in x_s.items():
                val = var.X
                if val > 0.01:
                    d = dist_s[(i,j)]
                    E_s += val * d * modes_info[m]['co2']
                    total_flow_s += val
                    if m in ['Van', 'Train']:
                        clean_flow_s += val
                    
                    # Print the row
                    print(f"{i:<15} {j:<15} {m:<8} {val:<12.1f} {int(d)}")
            
            print("-" * 65)
            
            alpha_s = clean_flow_s / total_flow_s if total_flow_s > 0 else 0.0
            
            results[name] = {
                "obj": model_s.objVal,
                "E": E_s,
                "alpha": alpha_s,
                "status": "Optimal"
            }
            print(f"RESULT: Optimal Cost = ${model_s.objVal:,.2f}")
            
        else:
            print("RESULT: Infeasible (Constraints too tight!)")
            results[name] = {
                "obj": 0, "E": 0, "alpha": 0, "status": "Infeasible"
            }

    # 5. Summary Table
    print("\n" + "="*80)
    print(f"{'Scenario':<15} {'Status':<12} {'Cost ($)':<15} {'Emissions (kg)':<18} {'Clean %':<10}")
    print("-" * 80)
    
    # Print Baseline
    print(f"{'Baseline':<15} {'Optimal':<12} {baseline_model.objVal:,.0f}{'':<4} {E0:,.0f}{'':<7} {alpha0:.1%}")
    
    for name, vals in results.items():
        if vals['status'] == 'Optimal':
            print(f"{name:<15} {vals['status']:<12} {vals['obj']:,.0f}{'':<4} {vals['E']:,.0f}{'':<7} {vals['alpha']:.1%}")
        else:
            print(f"{name:<15} {vals['status']:<12} -")
    print("="*80)

*****

## Method 2: Benders Decomp

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import time
import gc

def solve_benders(df_nodes, demand, modes_info, env, y_start=None, E_max=500000000, clean_quota=0.0, max_iter=50):
    """
    Solves Network Design using Benders Decomposition.
    SYNCHRONIZED with Method 1 (Trains > 500 miles).
    """
    start_time = time.time()
    
    # --- 1. DATA PREP ---
    nodes = df_nodes['City'].tolist()
    modes = list(modes_info.keys())
    clean_modes = ['Van', 'Train']
    
    valid_arcs_modes = []
    dist = {}
    costs = {}       
    emissions = {}   
    fixed_costs = {} 
    capacities = {}  
    
    for i in nodes:
        for j in nodes:
            if i == j: continue
            r1 = df_nodes[df_nodes['City'] == i].iloc[0]
            r2 = df_nodes[df_nodes['City'] == j].iloc[0]
            d_ij = get_dist(r1['Lat'], r1['Lon'], r2['Lat'], r2['Lon'])
            dist[(i,j)] = d_ij
            
            for mode in modes:
                # --- FIX: CONSISTENT 500 MILE CONSTRAINT ---
                if mode == 'Train' and d_ij < 500: 
                    continue
                
                valid_arcs_modes.append((i, j, mode))
                costs[(i,j,mode)] = modes_info[mode]['cost'] * d_ij
                emissions[(i,j,mode)] = modes_info[mode]['co2'] * d_ij
                fixed_costs[(i,j,mode)] = modes_info[mode]['fixed']
                capacities[(i,j,mode)] = modes_info[mode]['cap']

    # =========================================================================
    # MASTER PROBLEM (MP)
    # =========================================================================
    mp = gp.Model("Benders_Master", env=env)
    mp.Params.OutputFlag = 0 
    mp.Params.LazyConstraints = 1
    mp.Params.Threads = 1
    
    y = mp.addVars(valid_arcs_modes, vtype=GRB.BINARY, name="y")
    eta = mp.addVar(vtype=GRB.CONTINUOUS, lb=0, name="eta") 
    
    mp.setObjective(
        gp.quicksum(fixed_costs[i,j,m] * y[i,j,m] for i,j,m in valid_arcs_modes) + eta,
        GRB.MINIMIZE
    )
    
    # --- STRONG CAPACITY CUTS ---
    for i in nodes:
        if demand[i] > 0:
            out_expr = gp.quicksum(capacities[i,j,m] * y[i,j,m] for j in nodes for m in modes if (i,j,m) in valid_arcs_modes)
            mp.addConstr(out_expr >= demand[i], name=f"MinCap_Out_{i}")
        elif demand[i] < 0:
            in_expr = gp.quicksum(capacities[j,i,m] * y[j,i,m] for j in nodes for m in modes if (j,i,m) in valid_arcs_modes)
            mp.addConstr(in_expr >= abs(demand[i]), name=f"MinCap_In_{i}")

    # =========================================================================
    # SUBPROBLEM (SP) - PENALTY METHOD
    # =========================================================================
    sp = gp.Model("Benders_Subproblem", env=env)
    sp.Params.OutputFlag = 0 
    sp.Params.Threads = 1
    sp.Params.Method = 1
    
    x = sp.addVars(valid_arcs_modes, vtype=GRB.CONTINUOUS, name="x")
    slack = sp.addVars(nodes, vtype=GRB.CONTINUOUS, lb=0, name="slack")
    PENALTY_COST = 1e9 
    
    sp.setObjective(
        gp.quicksum(costs[i,j,m] * x[i,j,m] for i,j,m in valid_arcs_modes) + 
        gp.quicksum(PENALTY_COST * slack[i] for i in nodes), 
        GRB.MINIMIZE
    )
    
    # Constraints
    balance_constrs = {}
    for i in nodes:
        flow_out = gp.quicksum(x[i,j,m] for j in nodes for m in modes if (i,j,m) in valid_arcs_modes)
        flow_in  = gp.quicksum(x[j,i,m] for j in nodes for m in modes if (j,i,m) in valid_arcs_modes)
        
        if demand[i] > 0: 
             balance_constrs[i] = sp.addConstr(flow_out - flow_in >= demand[i] - slack[i])
        else: 
             balance_constrs[i] = sp.addConstr(flow_out - flow_in == demand[i] + slack[i])

    emissions_constr = sp.addConstr(
        gp.quicksum(emissions[i,j,m] * x[i,j,m] for i,j,m in valid_arcs_modes) <= E_max,
        name="Global_Emissions"
    )

    clean_expr = gp.quicksum(x[i,j,m] for i,j,m in valid_arcs_modes if m in clean_modes)
    total_expr = gp.quicksum(x[i,j,m] for i,j,m in valid_arcs_modes)
    quota_constr = sp.addConstr(clean_expr - clean_quota * total_expr >= 0, name="Quota")

    cap_constrs = {}
    for i,j,m in valid_arcs_modes:
        cap_constrs[i,j,m] = sp.addConstr(x[i,j,m] <= 0)

    # =========================================================================
    # EXPLICIT WARM START INJECTION
    # =========================================================================
    if y_start:
        print("Injecting Warm Start solution from Method 1...")
        count_seeded = 0
        
        # 1. Update Subproblem with Warm Start Capacities
        for i,j,m in valid_arcs_modes:
            if (i,j,m) in y_start:
                val = y_start[i,j,m].X
                if val > 0.5: # If active in Method 1
                    y[i,j,m].Start = 1
                    cap_constrs[i,j,m].RHS = capacities[i,j,m] * val
                    count_seeded += 1
            else:
                cap_constrs[i,j,m].RHS = 0
                y[i,j,m].Start = 0
        
        print(f"Seeded {count_seeded} active links from Method 1.")

        # 2. Solve Subproblem immediately to get the Cost
        sp.optimize()
        
        if sp.status == GRB.OPTIMAL:
            print(f"Warm Start SP Cost: ${sp.objVal:,.2f}")
            
            # 3. Add the "Golden Cut"
            pi = sp.getAttr('Pi', balance_constrs)
            mu = sp.getAttr('Pi', cap_constrs)
            lam = emissions_constr.Pi
            
            cut_expr = (
                sum(demand[i] * pi[i] for i in nodes) + 
                (E_max * lam) + 
                sum(capacities[i,j,m] * y[i,j,m] * mu[i,j,m] for i,j,m in valid_arcs_modes)
            )
            mp.addConstr(eta >= cut_expr, name="WarmStartCut")
            print("Warm Start Cut added.")

    # =========================================================================
    # LOOP
    # =========================================================================
    print(f"{'Iter':<5} {'Master Obj':<15} {'SP Obj':<15} {'Gap':<10} {'Status'}")
    print("-" * 65)
    
    upper_bound = float('inf')
    epsilon = 0.05 
    
    for k in range(1, max_iter + 1):
        mp.optimize()
        y_vals = mp.getAttr('X', y)
        eta_val = eta.X
        current_master_obj = mp.objVal
        
        # Update SP
        for i,j,m in valid_arcs_modes:
            cap_constrs[i,j,m].RHS = capacities[i,j,m] * y_vals[i,j,m]
        
        sp.optimize()
        
        if sp.status == GRB.OPTIMAL:
            sp_obj = sp.objVal
            slack_sum = sum(v.X for v in slack.values())
            is_feasible_network = (slack_sum < 0.1)
            
            true_total_cost = (current_master_obj - eta_val) + sp_obj
            
            if is_feasible_network:
                upper_bound = min(upper_bound, true_total_cost)
            
            lower_bound = current_master_obj
            gap = (upper_bound - lower_bound) / abs(upper_bound + 1e-10)
            
            gap_str = f"{gap:.2%}" if upper_bound < 1e15 else "Inf"
            status_str = "FEASIBLE" if is_feasible_network else "PENALTY"
            
            print(f"{k:<5} {current_master_obj:<15.2f} {sp_obj:<15.2f} {gap_str:<10} {status_str}")
            
            if is_feasible_network and sp_obj <= eta_val + epsilon * max(1, eta_val):
                print(f"Converged! Optimal Cost: ${upper_bound:,.2f}")
                break
            
            # Optimality Cut
            pi = sp.getAttr('Pi', balance_constrs)
            mu = sp.getAttr('Pi', cap_constrs)
            lam = emissions_constr.Pi
            
            cut_expr = (
                sum(demand[i] * pi[i] for i in nodes) + 
                (E_max * lam) + 
                sum(capacities[i,j,m] * y[i,j,m] * mu[i,j,m] for i,j,m in valid_arcs_modes)
            )
            mp.addConstr(eta >= cut_expr, name=f"OptCut_{k}")
            
        else:
            print(f"SP Failed: {sp.status}")
            break
        gc.collect()

    total_time = time.time() - start_time
    print(f"\nBenders Time: {total_time:.2f} seconds")
    return mp, y_vals, sp, dist

# =============================================================================
# RUN SEQUENCE
# =============================================================================

# 1. Generate Data 
df, dem, info = generate_data()
modes_info['Train']['cost'] = 1.80  # Dirty Baseline costs
modes_info['Truck']['cost'] = 2.20  

# 2. Get Method 1 Baseline Solution
print("\n--- Getting Warm Start from Method 1 ---")
# Ensure Method 1 ALSO has the 500 mile check in its logic if you haven't updated it yet,
# but assuming you just ran Method 1 successfully, we take the result:
model_baseline, x_sol, y_sol, distances = solve_logistics_network(
    df, dem, info, env, 
    E_max=5e8, clean_quota=0.0
)
print(f"Method 1 Cost: ${model_baseline.objVal:,.2f}")

# 3. Run Benders
print("\n--- Starting Method 2: Benders (Synchronized) ---")
master_model, y_final, sub_model, distances = solve_benders(
    df, dem, info, env,
    y_start=y_sol, 
    E_max=5e8, clean_quota=0.0
)

# 4. Final Result
if sub_model.status == GRB.OPTIMAL:
    print("\n" + "="*40)
    print(f"FINAL BENDERS RESULT: ${master_model.objVal:,.2f}")

*****